In [1]:
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load

verbose = False  # True prints more output
verbose = True  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [2]:
# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.
df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)

In [3]:
# number of max lookback tuples to create for iloc start_train:end_train:end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms 
n_samples = 2  
# n_samples = 100

# for training, the number of days to lookback from iloc max-lookback end_train
days_lookbacks = [30, 120, 60]

# number of days from end_train are used to evaluate effectiveness of the training
days_eval = 10  

# number of the most-common symbols from days_lookbacks' performance rankings to keep
# n_top_syms = 5
n_top_syms = 5  

syms_start = 0  #  start index of n_top_syms for evaluation
syms_end = 4  #  end index of n_top_syms for evaluation

In [4]:
# create a sets of iloc lookback slices (start_train:end_train:end_eval), where
# end_train - start_train = days_lookback
# end_eval - end_train = days_eval
# for example,
# if given:
#  n_samples = 2
#  days_lookbacks = [30, 60, 120]
#  days_eval = 10
# a possible result is:
#  max_lookback_slices:
#  [(150, 270, 280), (5, 125, 135)]
#  sets_lookback_slices:
#  [[(240, 270, 280), (210, 270, 280), (150, 270, 280)], [(95, 125, 135), (65, 125, 135), (5, 125, 135)]]  
max_lookback_slices = _3_random_slices(len_df_train, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'max_lookback_slices:\n{max_lookback_slices}')
  print(f'sets_lookback_slices:\n{sets_lookback_slices}')

max_lookback_slices:
[(90, 210, 220), (831, 951, 961)]
sets_lookback_slices:
[[(180, 210, 220), (150, 210, 220), (90, 210, 220)], [(921, 951, 961), (891, 951, 961), (831, 951, 961)]]


Generate lists, n_samples long, of the highest performance ranked symbols. The performance metrics are: CAGR/UI, CAGR/retnStd, retnStd/UI. n_top_syms of the best performing symbols from each metric are combined. The symbols are sorted by their number of appearances in the combined pool, and are placed in a list. A slice of the best performing symbols is selected by syms_start:syms_end, i.e. top_set_syms_n_freq[syms_start:syms_end].     

The sets_lookback_slices are iloc for training and evaluation dataframes. The sets is randomly generated.   
  



In [5]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    d_eval = end_eval - start_eval

    if verbose:
      print(f'days lookback: {lookback}')
      print(f'lb_slices:     {lb_slices}')
      print(f'lb_slice:      {lb_slice}')
      print(f'days eval:     {d_eval}')    
      print(f'start_train:   {start_train}')
      print(f'end_train:     {end_train}')
      # print(f'start_eval:    {start_eval}')
      # print(f'end_eval:      {end_eval}')`

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=n_top_syms)
    grp_most_common_syms.append(most_common_syms)
    
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_common_syms: {most_common_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_common_syms: {grp_most_common_syms}')
    # grp_most_common_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
  # get symbols from top_set_syms_n_freq

###################################  
  # top_set_syms = [i[0] for i in top_set_syms_n_freq]
  top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]  
###################################  
  
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {n_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {n_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {n_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

days lookback: 30
lb_slices:     [(180, 210, 220), (150, 210, 220), (90, 210, 220)]
lb_slice:      (180, 210, 220)
days eval:     10
start_train:   180
end_train:     210
perf_ranks: {'period-30': {'r_CAGR/UI': array(['MARA', 'ENPH', 'BOOT', 'OSTK', 'NOG'], dtype=object), 'r_CAGR/retnStd': array(['MARA', 'ENPH', 'BOOT', 'NOG', 'OSTK'], dtype=object), 'r_retnStd/UI': array(['H', 'ALGN', 'MRVL', 'DHI', 'NTAP'], dtype=object)}}
most_common_syms: [('MARA', 2), ('ENPH', 2), ('BOOT', 2), ('OSTK', 2), ('NOG', 2), ('H', 1), ('ALGN', 1), ('MRVL', 1), ('DHI', 1), ('NTAP', 1)]
+++ finish lookback slice 30 +++

days lookback: 60
lb_slices:     [(180, 210, 220), (150, 210, 220), (90, 210, 220)]
lb_slice:      (150, 210, 220)
days eval:     10
start_train:   150
end_train:     210
perf_ranks: {'period-60': {'r_CAGR/UI': array(['MARA', 'ENPH', 'GDS', 'OSTK', 'NSP'], dtype=object), 'r_CAGR/retnStd': array(['ENPH', 'MARA', 'GDS', 'OSTK', 'RCKT'], dtype=object), 'r_retnStd/UI': array(['ALGN', 'NTAP', 'N

In [6]:
print('z_grp_top_set_syms:')
z_grp_top_set_syms = zip(max_lookback_slices, grp_top_set_syms)
for item in z_grp_top_set_syms:
  print(item)
print('')

z_grp_top_set_syms:
((90, 210, 220), ['OSTK', 'ENPH', 'MARA', 'GDS'])
((831, 951, 961), ['LOB', 'SPWR', 'ALGN', 'GOGO'])



In [7]:
from yf_utils import _7_perf_eval
print('z_grp_top_set_syms:')
z_grp_top_set_syms = zip(max_lookback_slices, grp_top_set_syms)
# z_grp_top_set_syms = zip(max_lookback_slices, grp_top_set_syms[26:29])

for i, (_lookback_slice, _top_set_syms) in enumerate(z_grp_top_set_syms):
# for i, (_lookback_slice, _top_set_syms) in enumerate(z_grp_top_set_syms[26:29]):

  start_train = _lookback_slice[0]
  end_train = _lookback_slice[1]
  start_eval = end_train
  end_eval = _lookback_slice[2]

  print(f'{i + 1 } of {n_samples} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  d_start_train = df_train.index[start_train].strftime('%Y-%m-%d')
  d_end_train = df_train.index[end_train].strftime('%Y-%m-%d')
  d_start_eval = d_end_train
  d_end_eval = df_train.index[end_eval].strftime('%Y-%m-%d')
  d_df_eval_start = d_end_train
  d_df_eval_end = df_train.index[end_eval - 1].strftime('%Y-%m-%d')  
  print(f'max lookback dates: {d_start_train}, {d_end_train}, {d_end_eval}')
  print(f'df_eval dates (inclusive): {d_df_eval_start} - {d_df_eval_end}')    
  print(f'top_set_syms: {_top_set_syms}\n')

  df_eval = df_train[start_eval:end_eval][_top_set_syms]

  if verbose:
    # print(f'start_eval: {start_eval}')
    # print(f'end_eval:   {end_eval}')  
    print(f'start_eval: {start_eval},  date: {d_end_train}')
    print(f'end_eval:   {end_eval},  date: {d_end_eval},  df_eval last date: {d_df_eval_end}')      
    print(f'\ndf_eval:\n{df_eval}\n')


  _, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = _7_perf_eval(df_eval)
  print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]:>13,.3f}, {grp_retnStd_d_UI[1]:>13,.3f}, {grp_retnStd_d_UI[2]:>13,.3f}')
  print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_retnStd[1]:>13,.3f}, {grp_CAGR_d_retnStd[2]:>13,.3f}')
  print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]:>13,.3f}, {grp_CAGR_d_UI[1]:>13,.3f}, {grp_CAGR_d_UI[2]:>13,.3f}')

  _sym_idx = ['SPY']
  df_SPY = df_train[start_eval:end_eval][_sym_idx]
  _, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = _7_perf_eval(df_SPY)
  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI: {grp_retnStd_d_UI[0]:>13,.3f}, {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_UI[0]:>13,.3f}')
  print('='*50, '\n')  

z_grp_top_set_syms:
1 of 2 max_lookback_slice
max_lookback_slice: (90, 210, 220)
max lookback dates: 2017-06-08, 2017-11-28, 2017-12-12
df_eval dates (inclusive): 2017-11-28 - 2017-12-11
top_set_syms: ['OSTK', 'ENPH', 'MARA', 'GDS']

start_eval: 210,  date: 2017-11-28
end_eval:   220,  date: 2017-12-12,  df_eval last date: 2017-12-11

df_eval:
                 OSTK  ENPH       MARA        GDS
Date                                             
2017-11-28  57.599998  3.14  15.400000  20.120001
2017-11-29  52.000000  2.87  16.200001  19.600000
2017-11-30  47.099998  2.90  16.680000  20.450001
2017-12-01  42.299999  2.90  19.799999  20.450001
2017-12-04  46.099998  2.65  18.639999  20.129999
2017-12-05  45.849998  2.76  20.440001  19.780001
2017-12-06  42.650002  2.70  18.600000  17.820000
2017-12-07  45.650002  2.80  18.320000  18.209999
2017-12-08  45.080002  2.77  17.639999  18.639999
2017-12-11  55.000000  2.70  25.200001  19.549999

grp(retnStd/UI):   mean, std, mean/std:         1.052